In [ ]:
import bs4
bs4.__version__

In [ ]:
import sys
import csv
from selenium import webdriver
import time
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

In [ ]:
path_to_file = "/home/bil/Documents/Scraping/tripadvisor_2.csv"
num_page = 40
url = "https://www.tripadvisor.com/Restaurant_Review-g1380108-d17713871-Reviews-Gooseberry-Pecatu_Nusa_Dua_Peninsula_Bali.html"


driver = webdriver.Chrome('./chromedriver_linux64/chromedriver')
driver.get(url)
driver.maximize_window()

# Open the file to save the review
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

# change the value inside the range to save more or less reviews
for i in range(0, num_page):
    
    try:
        driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
    except:
        print("there is no read more element")
    
#     container = driver.find_elements_by_xpath(".//div[@class='review-container']")
    page_soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews_container = page_soup.find_all('div', class_='review-container')
    time.sleep(20)
    
    for j in range(len(reviews_container)):
        
        user = reviews_container[j].find_all('div', class_="memberOverlayLink clickable")
        UID = user[0].get('id')
        user_name = user[0].find_all('div')[-1].get_text()
        count_review = user[1].find_all('span')[-1].get_text()
        
        re = reviews_container[j].find('div', class_="ui_column is-9")

        title = re.find('span', class_='noQuotes').get_text()
        date = re.find('span', class_='ratingDate').get('title')
        rating = re.find('span', class_='ui_bubble_rating').get('class')[1].split('_')[1]
        review = re.find('p', class_='partial_entry').get_text()

        csvWriter.writerow([UID, user_name, count_review,date, rating, title, review]) 

    # change the page
    try:
        driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()
    except:
        driver.close()
driver.close()